In [1]:
import pandas as pd
import gzip
import json
import seaborn as sns
import numpy as np
import time
import html
import re
import os
from typing import *
import urllib.request
from PIL import Image
import cv2 as cv
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
tqdm.pandas()

### Вторая фаза предобработки

In [2]:
%%time
metadata_test = pd.read_csv('metadata_parsed_with_images.csv')

CPU times: total: 8.61 s
Wall time: 8.63 s


In [3]:
metadata_test = metadata_test.iloc[500000:600000]
# metadata_test = metadata_test.iloc[0:300000]

In [4]:
metadata_test.shape

(100000, 12)

In [5]:
metadata_test = metadata_test.fillna('')

In [6]:
import ast

In [7]:
%%time
metadata_test['images'] = metadata_test['images'].apply(lambda x: ast.literal_eval(x))

CPU times: total: 1.33 s
Wall time: 1.35 s


In [8]:
%%time
metadata_test['imagesHighRes'] = metadata_test['imagesHighRes'].apply(lambda x: ast.literal_eval(x))

CPU times: total: 1.27 s
Wall time: 1.27 s


In [9]:
part = metadata_test.copy()

In [10]:
import traceback
from functools import partial

In [14]:
# import psutil, humanize

# def mem_report():
#     print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))

# mem_report()

In [15]:
part.head()

,rank,category,description,title,brand,feature,price,asin,images,imagesHighRes,category_id,rank_scaled,image_count
500000,7326.0,Electronics > Accessories & Supplies > Televis...,This is a custom built replacement remote made...,Replacement Remote Control for PROSCAN 40LC45S...,Redi-Remote,Redi-Remotes cannot be programmed to control a...,2.00,B009H5LHZ6,[],[],854,2.135130,0
500001,9511.0,Electronics > Televisions & Video,This is a custom built replacement remote made...,Replacement Remote Control for PROSCAN 40LC45S...,Redi-Remote,Redi-Remotes cannot be programmed to control a...,2.00,B009H5LHZ6,[],[],920,2.021770,0
500002,17398.0,Electronics > Accessories & Supplies > Audio &...,This is a custom built replacement remote made...,Replacement Remote Control for HAFLER SERIES 945,Redi-Remote,Redi-Remotes cannot be programmed to control a...,29.95,B009H5L9W2,[],[],1015,1.759497,0
500003,40080.0,Electronics > Accessories & Supplies > Televis...,This is a custom built replacement remote made...,Replacement Remote Control for HAFLER SERIES 945,Redi-Remote,Redi-Remotes cannot be programmed to control a...,29.95,B009H5L9W2,[],[],854,1.397069,0
500004,57785.0,Electronics > Televisions & Video,This is a custom built replacement remote made...,Replacement Remote Control for HAFLER SERIES 945,Redi-Remote,Redi-Remotes cannot be programmed to control a...,29.95,B009H5L9W2,[],[],920,1.238181,0


In [16]:
metadata_test.shape

(100000, 13)

In [17]:
import traceback

def image_url(urls, path='images_0/'):
    import traceback
    try:
        if len(urls) == 0:
            return ''
        url = urls[0]
        return path + url[49:]
    except Exception:
        print(f'{traceback.format_exc()}')
        return ''

In [18]:
metadata_test['image_count'] = metadata_test.images.apply(lambda x: len(x))

In [19]:
parts = []

part = metadata_test.copy()
part['image_url_high_res'] = part.imagesHighRes.apply(partial(image_url, path=f'images_high_res_small/'))
parts.append(part)
    
df = pd.concat(parts)

In [20]:
# parts = []

# for i in trange(3):
#     l = i * 100000
#     r = min(l + 100000, metadata_test.shape[0])
#     print(f"[{l}:{r}]")
#     mem_report()
#     part = metadata_test.iloc[l:r]
#     print('images')
#     part['image_url'] = part.images.apply(partial(image_url, path=f'images_{i}/'))
#     print('images high res')
#     part['image_url_high_res'] = part.imagesHighRes.apply(partial(image_url, path=f'images_high_res_{i}/'))
#     parts.append(part)
    
# df = pd.concat(parts)

Разобъем все товары с одинаковыми категориями на батчи  по 40 товаров, у каждого батча уникальный category_id

In [22]:
df = df.sort_values(by='category_id')

In [23]:
df['category_rank'] = df.groupby(['category_id']).cumcount()

In [24]:
df['category_id_batched'] = df.category_rank.apply(lambda x: x // 40)

In [25]:
df['category_id'] = (df['category_id'] + 1) * 10000 + df['category_id_batched'] 

In [26]:
len(set(df.category_id))

2943

In [27]:
metadata_parsed = df.copy()

In [29]:
def remove_seetop100(x):
    start = x.lower().find('see top 100') 
    if start != -1:
        return x[:start - 1]
    return x

In [30]:
metadata_parsed['category'] = metadata_parsed['category'].apply(lambda x: remove_seetop100(x))

In [31]:
metadata_parsed

,rank,category,description,title,brand,feature,price,asin,images,imagesHighRes,category_id,rank_scaled,image_count,image_url_high_res,category_rank,category_id_batched
595306,167634.0,Home & Kitchen,"""Brand New, High Speed USB 2.0 Data/Lead Cable...",maxinbuy MaxLLTo 5FT Extra Long USB DC Power C...,maxinbuy,USB DC Power Charging + Data Cable/Cord/Lead F...,6.99,B00J4LD8PK,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,0.775634,5,images_high_res_small/41u0Pzcv8EL.jpg,0,0
565911,249287.0,Home & Kitchen,"The Hampton nautical solid Brass 62"" binocular...",Hampton Nautical Floor Standing Admirals Solid...,Hampton Nautical,"12x magnification ultra large image size, Soli...",321.95,B00GA204XU,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,0.603297,6,images_high_res_small/412%2BeCI5TQL.jpg,1,0
524672,562727.0,Home & Kitchen,These are high quality 4 ply picture mats cut ...,Black 16x20 Collage Picture Mat for 15 Photos ...,bux1 picture matting,"All openings are reduced by 1/2"" to overlap th...",10.99,B00BWUDNNS,[],[],10000,0.249699,0,,2,0
567321,179494.0,Home & Kitchen,"""- ICC Handset cord- 12 length- Curly handset ...",Cablesys GCHA444012-FCR / 12 RED Handset Cord,Cablesys,"Manufactured By.: ICC, 1200RD",4.92,B00GG394NE,[],[],10000,0.745947,0,,3,0
594030,102515.0,Home & Kitchen,Smart magnetic cover For iPad 2/3/4 Please not...,"Smart Magnetic Cover for Apple iPad 2, iPad 3 ...",inShang,inShang Super Smart Magnetic cover + back case...,8.98,B00IZX2NBS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,0.989209,13,images_high_res_small/41y-LE389NL.jpg,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575168,48.0,Electronics > Accessories & Supplies > Audio &...,Accessories ONLY! The laptop is not included h...,uxcell Repair Part Internal Speakers Magnet 30...,uxcell,Product Name : Laptop Internal Speaker;Fit for...,4.63,B00H8TDAK8,[],[],24640000,4.318755,0,,23,0
535688,271.0,Electronics > Accessories & Supplies > Audio &...,"CUSTOM MADE Aftermarket 2"" Kicker Voice Coil. ...","2"" Kicker Voice Coil - D4 Ohm - L7 L5 CVR Solo...",Springfield Speaker,"Not OEM; Custom-made for the application, Blac...",2.00,B00CUJ29A8,[],[],24640000,3.567027,0,,24,0
570870,249.0,Electronics > Accessories & Supplies > Audio &...,This magnet speaker is designed with aluminum ...,uxcell 57mm Dia Silver Tone Round External Mag...,uxcell,Product Name : Magnet Speaker;Power : 0.5W;Res...,3.83,B00GSVI2C8,[],[],24640000,3.603797,0,,25,0
521267,242.0,Electronics > Accessories & Supplies > Audio &...,This magnet speaker is designed with aluminum ...,uxcell 0.5W 0.5 Watt 8 Ohm 50mm Dia Aluminum R...,uxcell,Product Name : Magnet Speaker;Power : 0.5W;Res...,3.98,B00BM1QSM0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,24640000,3.616181,1,images_high_res_small/41G16Ej%2B5pL.jpg,26,0


### Посчитаем NDCG случайного предсказания

Для начала нужно понять, что такое случайное предсказание. Модель будет предсказывать новые ранги, а не просто сортировать уже имеющиеся.  
В качестве довольно плохого приближения можно исплользовать равномерное распределение на [0, 6]  
Более приближенное распределение рангов получится, если выбирать случайные ранги из списка всех рангов

In [32]:
category_ids = set(list(metadata_parsed.category_id))

In [33]:
all_ranks = list(metadata_parsed["rank_scaled"])

In [34]:
print(len(all_ranks))

100000


In [35]:
import random

NUM_ITEMS = 40

real_ranks = []
random_ranks = []

for category in category_ids:
    ordered_rank_list = list(metadata_parsed[metadata_parsed['category_id'] == category].sort_values(by='rank').iloc[0:NUM_ITEMS]['rank_scaled'])
    if len(a) < 40:
        continue
    real_ranks.append(ordered_rank_list)
    random_ranks.append(random.sample(all_ranks, NUM_ITEMS))

In [36]:
print(np.array(real_ranks).shape)
print(np.array(random_ranks).shape)

(2162, 40)
(2162, 40)


In [37]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

C:\Users\artyo\anaconda3\envs\tfr-bert-demo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
for i in range(1, 41):
    ndcg = tfr.keras.metrics.NDCGMetric(topn=i)
    print(f"NDCG@{i}: {ndcg(real_ranks, random_ranks).numpy()}")

NDCG@1: 0.31944823265075684
NDCG@2: 0.3506602644920349
NDCG@3: 0.37423399090766907
NDCG@4: 0.39595767855644226
NDCG@5: 0.41344648599624634
NDCG@6: 0.43023455142974854
NDCG@7: 0.4457526206970215
NDCG@8: 0.4602159857749939
NDCG@9: 0.4726872146129608
NDCG@10: 0.48477351665496826
NDCG@11: 0.4971722364425659
NDCG@12: 0.5081313848495483
NDCG@13: 0.5196612477302551
NDCG@14: 0.5306726694107056
NDCG@15: 0.5415663719177246
NDCG@16: 0.5518699884414673
NDCG@17: 0.5616085529327393
NDCG@18: 0.5713207125663757
NDCG@19: 0.5808902978897095
NDCG@20: 0.5905998945236206
NDCG@21: 0.5996828079223633
NDCG@22: 0.6085206270217896
NDCG@23: 0.6169990301132202
NDCG@24: 0.6260000467300415
NDCG@25: 0.6350152492523193
NDCG@26: 0.643807590007782
NDCG@27: 0.6525022983551025
NDCG@28: 0.6610872745513916
NDCG@29: 0.6696141362190247
NDCG@30: 0.6781430244445801
NDCG@31: 0.686686635017395
NDCG@32: 0.6947803497314453
NDCG@33: 0.7029333710670471
NDCG@34: 0.711721658706665
NDCG@35: 0.7200694680213928
NDCG@36: 0.728083372116088

Для всего датасета:

In [62]:
for i in range(1, 41):
    ndcg = tfr.keras.metrics.NDCGMetric(topn=i)
    print(f"NDCG@{i}: {ndcg(real_ranks, random_ranks).numpy()}")

NDCG@1: 0.27741721272468567
NDCG@2: 0.303310364484787
NDCG@3: 0.32507145404815674
NDCG@4: 0.3434201180934906
NDCG@5: 0.3600502014160156
NDCG@6: 0.3761254549026489
NDCG@7: 0.39083534479141235
NDCG@8: 0.4047471880912781
NDCG@9: 0.4179899990558624
NDCG@10: 0.43005138635635376
NDCG@11: 0.44192999601364136
NDCG@12: 0.4535923898220062
NDCG@13: 0.4647643268108368
NDCG@14: 0.4757555425167084
NDCG@15: 0.48630157113075256
NDCG@16: 0.4966924786567688
NDCG@17: 0.5067606568336487
NDCG@18: 0.5164785385131836
NDCG@19: 0.5263077020645142
NDCG@20: 0.5360348224639893
NDCG@21: 0.5454726219177246
NDCG@22: 0.5549251437187195
NDCG@23: 0.5642771124839783
NDCG@24: 0.5732648968696594
NDCG@25: 0.5825837850570679
NDCG@26: 0.5915178060531616
NDCG@27: 0.60047447681427
NDCG@28: 0.6096016764640808
NDCG@29: 0.6185185313224792
NDCG@30: 0.6272718906402588
NDCG@31: 0.636314332485199
NDCG@32: 0.6452332735061646
NDCG@33: 0.6542215943336487
NDCG@34: 0.663281261920929
NDCG@35: 0.6723036170005798
NDCG@36: 0.6814430356025696


Подмешиваем какой-то процент случайных рангов в настоящие, считаем NDCG

In [39]:
real_ranks = np.array(pd.DataFrame(real_ranks).sample(frac=1))

In [40]:
random_ranks = np.array(pd.DataFrame(random_ranks).sample(frac=1))

In [41]:
print(np.array(real_ranks).shape)
print(np.array(random_ranks).shape)

(2162, 40)
(2162, 40)


In [42]:
import random
percentage = [0., 10., 20., 25., 30., 50., 100.]

for p in percentage:
    target_ranks = []
    predicted_ranks = []
    for i, (real, rand) in enumerate(zip(real_ranks, random_ranks)): 
        for i in range(10):  
            nice = [] 
            for g, b in zip(real_ranks[(i + 1) // len(real_ranks)], rand):
                n = random.random()    
                if n >= p / 100.:
                    nice.append(g)
                else:
                    nice.append(b) 
            predicted_ranks.append(nice)
            target_ranks.append(real)

    ndcg = tfr.keras.metrics.NDCGMetric()
    print(f"Percentage: {p}")
    for i in [1, 3, 5, 10, 40]:
        ndcg = tfr.keras.metrics.NDCGMetric(topn=i)
        print(f"NDCG@{i}: {ndcg(target_ranks, predicted_ranks).numpy()}")

Percentage: 0.0
NDCG@1: 1.0
NDCG@3: 1.0
NDCG@5: 1.0
NDCG@10: 1.0
NDCG@40: 1.0
Percentage: 10.0
NDCG@1: 0.9030899405479431
NDCG@3: 0.926919162273407
NDCG@5: 0.9280373454093933
NDCG@10: 0.9419649243354797
NDCG@40: 0.9709725379943848
Percentage: 20.0
NDCG@1: 0.8128920793533325
NDCG@3: 0.8536324501037598
NDCG@5: 0.8589550852775574
NDCG@10: 0.8848013281822205
NDCG@40: 0.9425324201583862
Percentage: 25.0
NDCG@1: 0.7711101174354553
NDCG@3: 0.8189769387245178
NDCG@5: 0.826300323009491
NDCG@10: 0.8572250008583069
NDCG@40: 0.9293316602706909
Percentage: 30.0
NDCG@1: 0.7301534414291382
NDCG@3: 0.7832774519920349
NDCG@5: 0.7936449646949768
NDCG@10: 0.8292310833930969
NDCG@40: 0.9156654477119446
Percentage: 50.0
NDCG@1: 0.5819336175918579
NDCG@3: 0.6533886790275574
NDCG@5: 0.6731239557266235
NDCG@10: 0.7238375544548035
NDCG@40: 0.8658838868141174
Percentage: 100.0
NDCG@1: 0.3173888325691223
NDCG@3: 0.3737831711769104
NDCG@5: 0.4128538966178894
NDCG@10: 0.4847838878631592
NDCG@40: 0.7634375095367432

Для всего датасета

In [40]:
import random
percentage = [0., 10., 20., 25., 30., 50., 100.]

for p in percentage:
    target_ranks = []
    predicted_ranks = []
    for i, (real, rand) in enumerate(zip(real_ranks, random_ranks)): 
        for i in range(10):  
            nice = [] 
            for g, b in zip(real_ranks[(i + 1) // len(real_ranks)], rand):
                n = random.random()    
                if n >= p / 100.:
                    nice.append(g)
                else:
                    nice.append(b) 
            predicted_ranks.append(nice)
            target_ranks.append(real)

    ndcg = tfr.keras.metrics.NDCGMetric()
    print(f"Percentage: {p}")
    for i in [1, 3, 5, 10, 40]:
        ndcg = tfr.keras.metrics.NDCGMetric(topn=i)
        print(f"NDCG@{i}: {ndcg(target_ranks, predicted_ranks).numpy()}")

Percentage: 0.0
NDCG@1: 1.0
NDCG@3: 1.0
NDCG@5: 1.0
NDCG@10: 1.0
NDCG@40: 1.0
Percentage: 10.0
NDCG@1: 0.7496519684791565
NDCG@3: 0.834526002407074
NDCG@5: 0.8668624758720398
NDCG@10: 0.8979381322860718
NDCG@40: 0.9390802383422852
Percentage: 20.0
NDCG@1: 0.5768909454345703
NDCG@3: 0.7048552632331848
NDCG@5: 0.7554416060447693
NDCG@10: 0.8117515444755554
NDCG@40: 0.8898542523384094
Percentage: 25.0
NDCG@1: 0.5121918320655823
NDCG@3: 0.649610161781311
NDCG@5: 0.7060869932174683
NDCG@10: 0.772880494594574
NDCG@40: 0.8685073256492615
Percentage: 30.0
NDCG@1: 0.4576380252838135
NDCG@3: 0.5997496247291565
NDCG@5: 0.6610116362571716
NDCG@10: 0.7365870475769043
NDCG@40: 0.8490076661109924
Percentage: 50.0
NDCG@1: 0.3358527719974518
NDCG@3: 0.45553356409072876
NDCG@5: 0.5236450433731079
NDCG@10: 0.6170800924301147
NDCG@40: 0.7898462414741516
Percentage: 100.0
NDCG@1: 0.2747800052165985
NDCG@3: 0.3228565752506256
NDCG@5: 0.35969698429107666
NDCG@10: 0.4283210337162018
NDCG@40: 0.718034625053405

### Разбиваем на train и test

In [66]:
from sklearn.model_selection import GroupShuffleSplit

In [67]:
gss = GroupShuffleSplit(test_size=.33, n_splits=1, random_state = 7).split(metadata_parsed, groups=metadata_parsed['category_id'])

X_train_inds, X_test_inds = next(gss)

train_data = metadata_parsed.iloc[X_train_inds]
test_data = metadata_parsed.iloc[X_test_inds]

In [68]:
# train_data.to_csv('train_with_images.csv', index=False)
train_data.to_csv('train_with_images_small.csv', index=False)

In [69]:
# test_data.to_csv('test_with_images.csv', index=False)
test_data.to_csv('test_with_images_small.csv', index=False)

In [70]:
# test_data = pd.read_csv('test_with_images.csv')
test_data = pd.read_csv('test_with_images_small.csv')

In [71]:
test_data.shape

(16654, 16)

### Отдельно eval-датасет: отличается от test только тем, что товары лежат сразу в отсортированном по рангу виде, для удобства 

In [72]:
len(set(test_data.category_id))

581

In [73]:
eval_categories = []

for i in set(test_data.category_id):
    l = test_data[test_data['category_id'] == i].sort_values(by='rank').sample(frac=1)
    assert l.shape[0] <= 40
    eval_categories.append(l)    
eval_data = pd.concat(eval_categories)

In [74]:
len(set(eval_data.category_id))

581

In [75]:
eval_data.shape

(16654, 16)

In [76]:
# eval_data.to_csv('eval_with_images.csv', index=False)
eval_data.to_csv('eval_with_images_small.csv', index=False)

In [77]:
eval_data.head()

,rank,category,description,title,brand,feature,price,asin,images,imagesHighRes,category_id,rank_scaled,image_count,image_url_high_res,category_rank,category_id_batched
5992,158413.0,Electronics > Accessories & Supplies > Cables,"""Due to popular demand, Mimo Monitors has desi...",Mimo Monitors USB/power cable - 15 for P/N: TO...,Mimo,"Cable Features: Molded, Length: 15 ft., Connec...",19.99,B00CEHKM7I,[],[],8620032,0.800206,0,NaN,1293,32
5996,166711.0,Electronics > Accessories & Supplies > Cables,This professional quality AIR802 CA195 antenna...,"AIR802 CA195 Black Cable, RP-SMA Plug-Male to ...",AIR802,"AIR802 CA195 Cable (Equivalent to LMR 195), RP...",13.05,B009S8ZY8S,[],[],8620032,0.778032,0,NaN,1297,32
5993,115020.0,Electronics > Accessories & Supplies > Cables,Quantity: 4pcs\nTitanize treatment for the sur...,Vktech 4PCS RCA Plug Audio Video Locking Cable...,Vktech,"RCA plug connector for audio video cable, Prof...",3.89,B00D5Q7X9M,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,8620032,0.939223,6,images_high_res_small/413kXr0fkDL.jpg,1294,32
6018,108795.0,Electronics > Accessories & Supplies > Cables,"""The dCables® USB Cable for the Canon PowerSho...",dCables Canon PowerShot ELPH 130 IS USB Cable ...,dCables,Use this cable to download images from the Can...,5.75,B00B00UCTI,[],[],8620032,0.963388,0,NaN,1319,32
6002,204238.0,Electronics > Accessories & Supplies > Cables,Keystone Snap-in Cat.6 couplers provide a full...,RJ45 Female to Female UTP CAT.6 Keystone Coupl...,iMBAPrice,For use with Cat.6 Ethernet cables / Backward ...,77.99,B00BJP18MY,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,8620032,0.689860,4,images_high_res_small/21LLjk25IdL.jpg,1303,32
